In [3]:
ord('有')

26377

In [27]:
import io
import json
import requests
import itertools

url = 'http://hmi.chengjiukehu.com/g2p-pp-service'
headers = {
        'Content-Type': 'application/json'
    }

data = {
    'user_id': '',
    'request_id': '974d59c5-34b8-4635-a797-0fe4f3c030a0',
    'session_id': '',
    'trace_id': '',
    'span_id': '',
    'text': 'asciimath 分 ascii math'
}
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    res = json.loads(response.text)
    phonemes, prosodies = res['phonemes'], res['prosodies']
    phonemes = list(itertools.chain(*phonemes))
    prosodies = list(itertools.chain(*prosodies))
    indices = sorted([index for index, value in enumerate(phonemes) if value == 'xer'], reverse=True)
    #for index in indices:
    #    phonemes[index] = 'er2'
print(phonemes)
print(prosodies)

['AH0', 'S', 'IH1', 'CH', 'AH0', 'M', 'sp3', 'f', 'en1', 'sp2', 'AE1', 'S', 'K', 'IY0', 'M', 'AE1', 'TH']
['0', '0', '0', '0', '0', '#3', '0', '0', '#2', '0', '0', '0', '0', '#1', '0', '0', '#4']


In [2]:
import io
import time
import numpy as np
from pydub import AudioSegment
import soundfile as sf

def convert_audio(audio_data, sample_rate=24000, bit_depth=16, channels=1, format='mp3'):
    # 将NDArray转换为指定位深格式
    if bit_depth == 16:
        audio_data = np.asarray(audio_data * 32767, dtype=np.int16)
    elif bit_depth == 24:
        audio_data = np.asarray(audio_data * 8388607, dtype=np.int32)
    elif bit_depth == 32:
        audio_data = np.asarray(audio_data * 2147483647, dtype=np.int32)
    else:
        raise ValueError("Unsupported bit depth. Please use 16, 24, or 32.")
    
    if format == 'wav':
        audio_segment = AudioSegment(
            audio_data.tobytes(), 
            frame_rate=sample_rate,
            sample_width=audio_data.dtype.itemsize, 
            channels=channels
        )
        wav_data = audio_segment.export(format=format, bitrate='64k')
        return wav_data.read()
    elif format == 'pcm':
        return audio_data.tobytes()
    elif format == 'mp3':
        audio_segment = AudioSegment(
            audio_data.tobytes(), 
            frame_rate=sample_rate,
            sample_width=audio_data.dtype.itemsize, 
            channels=channels
        )
        mp3_data = audio_segment.export(format=format, bitrate='64k')
        return mp3_data.read()
    else:
        raise ValueError("Unsupported format. Please use 'wav', 'pcm', or 'mp3'.")
    
def convert_audio_tobyte(data, format='mp3'):
        if format == 'wav':
            audio_buffer = io.BytesIO()
            audio_buffer.write(data)
            return audio_buffer.getvalue()

        elif format == 'pcm':
            # with open(filename, 'wb') as pcm_file:
            #     pcm_file.write(data)
            audio_buffer = io.BytesIO()
            audio_buffer.write(data)
            return audio_buffer.getvalue()
        elif format == 'mp3':
            # with open(filename, 'wb') as mp3_file:
            #     mp3_file.write(data)
            audio_buffer = io.BytesIO()
            audio_buffer.write(data)
            return audio_buffer.getvalue()
        else:
            raise ValueError(
                "Unsupported format. Please use 'wav', 'pcm', or 'mp3'.")

In [21]:
import subprocess

pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.int16)

audio_data = np.asarray(audio_data * 32767, dtype=np.int16)
audio_segment = AudioSegment(
    audio_data.tobytes(), 
    frame_rate=24000,
    sample_width=audio_data.dtype.itemsize, 
    channels=1
)
wav_data = audio_segment.export(format='wav', bitrate='64k')
wav_data = wav_data.read()
    
time1 = time.time()
subprocess.run(['lame', '--silent', wav_data, 'output.mp3'])
time2 = time.time()
print('pcm 转 wav 耗时：', (time2 - time1)*1000,'ms')

res = convert_audio_tobyte(wav_data, format='mp3')
time3 = time.time()
print('mp3 转 byte耗时：',(time3 - time2)*1000,'ms')

with open('test_data/audio/time_test.mp3', 'wb') as f:
    f.write(wav_data)
time4 = time.time()
print('mp3 存储到本地耗时：',(time4 - time3)*1000,'ms') 

ValueError: embedded null byte

In [3]:
import subprocess

pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.int16)

time1 = time.time()
subprocess.run(['ffmpeg', '-y', '-f', 's16le', '-ar', '44100', '-ac', '1', '-i', pcm_file_path, 'test_data/audio/time_test.mp3'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time2 = time.time()
print('ffmpge 转 pcm 为 mp3 并存储到本地耗时',(time2 - time1)*1000,'ms')


ffmpge 转 pcm 为 mp3 并存储到本地耗时 73.58098030090332 ms


In [36]:
import subprocess
import numpy as np
import time

In [46]:
pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.int16)

pcm_byte_stream = audio_data.tobytes()

time1 = time.time()
result = subprocess.run(['ffmpeg', '-y', '-f', 's16le', '-ar', '24000', '-ac', '1', '-i', 'pipe:0', '-f', 'mp3', 'pipe:1'],
                        input=pcm_byte_stream, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
mp3_byte_stream = result.stdout
time2 = time.time()
print('ffmpeg 转 pcm 为 mp3 并获取字节流耗时', (time2 - time1) * 1000, 'ms')

ffmpeg 转 pcm 为 mp3 并获取字节流耗时 64.06450271606445 ms


In [3]:
import numpy as np
import subprocess
import time
from pydub import AudioSegment
import io

# 假设你的 PCM 文件路径
pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.float32)

audio_data = np.asarray(audio_data * 32767, dtype=np.int16)

total_time, total_time_2 = 0, 0
for _ in range(100):
    
    pcm2wav_time1 = time.time()
    audio_segment = AudioSegment(
    audio_data.tobytes(), 
    frame_rate=24000,
    sample_width=audio_data.dtype.itemsize, 
    channels=1
    )
    
    wav_io = io.BytesIO()
    audio_segment.export(wav_io, format='wav', bitrate='64k')
    wav_io.seek(0)
    pcm2wav_time2 = time.time()
    total_time_2 += (pcm2wav_time2 - pcm2wav_time1) * 1000 

    # 使用 subprocess 调用 lame，将 WAV 数据通过管道传递并获取 MP3 字节流
    time1 = time.time()
    process = subprocess.Popen(
        ['lame', '--silent', '-', '-'],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    # 将 WAV 字节流写入 stdin 并从 stdout 读取 MP3 字节流
    mp3_byte_stream, _ = process.communicate(input=wav_io.read())
    time2 = time.time()
    total_time += (time2 - time1) * 1000 

average_time = total_time / 100
print('平均耗时：', average_time, 'ms')

average_time = total_time_2 / 100
print('平均耗时：', average_time, 'ms')

/tmp/ipykernel_1452928/1437504894.py:11: RuntimeWarning: overflow encountered in multiply
  audio_data = np.asarray(audio_data * 32767, dtype=np.int16)
/tmp/ipykernel_1452928/1437504894.py:11: RuntimeWarning: invalid value encountered in multiply
  audio_data = np.asarray(audio_data * 32767, dtype=np.int16)
/tmp/ipykernel_1452928/1437504894.py:11: RuntimeWarning: invalid value encountered in cast
  audio_data = np.asarray(audio_data * 32767, dtype=np.int16)


平均耗时： 26.34507417678833 ms
平均耗时： 0.048623085021972656 ms


In [4]:
import numpy as np
import subprocess
import time
from pydub import AudioSegment
import io

# 假设你的 PCM 文件路径
pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.float32)

audio_data = np.asarray(audio_data * 32767, dtype=np.int16)

total_time, total_time_2 = 0, 0
for _ in range(100):
    
    pcm2wav_time1 = time.time()
    
    audio_segment = AudioSegment(
    audio_data.tobytes(), 
    frame_rate=24000,
    sample_width=audio_data.dtype.itemsize, 
    channels=1
    )
    
    wav_io = io.BytesIO()
    audio_segment.export(wav_io, format='mp3', bitrate='64k')
    wav_io.seek(0)
    pcm2wav_time2 = time.time()
    total_time_2 += (pcm2wav_time2 - pcm2wav_time1) * 1000 

average_time = total_time_2 / 100
print('平均耗时：', average_time, 'ms')

/tmp/ipykernel_1452928/3777487097.py:11: RuntimeWarning: overflow encountered in multiply
  audio_data = np.asarray(audio_data * 32767, dtype=np.int16)
/tmp/ipykernel_1452928/3777487097.py:11: RuntimeWarning: invalid value encountered in multiply
  audio_data = np.asarray(audio_data * 32767, dtype=np.int16)
/tmp/ipykernel_1452928/3777487097.py:11: RuntimeWarning: invalid value encountered in cast
  audio_data = np.asarray(audio_data * 32767, dtype=np.int16)


平均耗时： 54.166104793548584 ms


In [1]:
import ffmpeg
import numpy as np
import time

pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.int16)

pcm_byte_stream = audio_data.tobytes()

total_time = 0

# 循环100次，计算平均耗时
for _ in range(100):
    # 使用 ffmpeg 将 PCM 数据转换为 MP3 字节流
    process = (
        ffmpeg
        .input('pipe:0', format='s16le', ar=24000, ac=1)
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    time1 = time.time()
    
    # 向 ffmpeg 进程写入 PCM 数据并获取输出
    out, err = process.communicate(input=pcm_byte_stream)
    
    time2 = time.time()

    total_time += (time2 - time1) * 1000  # 以毫秒为单位

average_time = total_time / 100
print('ffmpeg 转 pcm 为 wav 平均耗时', average_time, 'ms')

# 获取 MP3 字节流
mp3_byte_stream = out

ffmpeg 转 pcm 为 wav 平均耗时 44.66773509979248 ms


In [32]:
import subprocess
import numpy as np
import time

# 读取 PCM 文件
pcm_file_path = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend/vits/poem/tts_audios/96_neutral_14_10_18.pcm'
audio_data = np.fromfile(pcm_file_path, dtype=np.int16)

pcm_byte_stream = audio_data.tobytes()

total_time = 0

for _ in range(100):
    time1 = time.time()
    result = subprocess.run(
        ['ffmpeg', '-y', '-f', 's16le', '-ar', '24000', '-ac', '1', '-i', 'pipe:0', '-f', 'mp3', 'pipe:1'],
        input=pcm_byte_stream,
        stdout=subprocess.PIPE,
        stderr=subprocess.DEVNULL
    )
    time2 = time.time()
    total_time += (time2 - time1) * 1000  

average_time = total_time / 100  # 计算平均耗时
print('ffmpeg 转 pcm 为 mp3 并获取字节流平均耗时', average_time, 'ms')

mp3_byte_stream = result.stdout


ffmpeg 转 pcm 为 mp3 并获取字节流平均耗时 61.42758131027222 ms


In [15]:
import re

input = '<speak voice_type=\"xiaosi\" emotion=\"happy\" volume=\"0.9\" rate=\"1.1\" pitch=\"1.0\"> 你好啊,今天天气</speak>'

if re.search(r'(<speak.*?>)(.*?)(</speak>)', input, re.DOTALL):
    print('yes')

yes


In [12]:
re.findall(r'(<speak.*?>)(.*?)(</speak>)', input)

[]

In [19]:
import uuid

uuid.uuid4()

UUID('84913546-88a0-45b4-aad3-bc65efc7cc9c')

In [22]:
from tal_frontend.frontend.normalizer.cn.cn_normalizer import cn_Normalizer as cn_Normalizer
from tal_frontend.frontend.normalizer.en.en_normalizer import en_normalize

zh_normalizer = cn_Normalizer()
en_normalizer = en_normalize

In [6]:
text = "it's a good day good-looking boy it''s good--looking"
for _ in range(100):
    en_normalizer(text)

In [32]:
zh_normalizer.normalize('题目给出了一次函数y等于k x加b，给定了两个点（1,1）和（2,-4），这两个点都在函数上， -1 1+1=2')

'题目给出了一次函数y等于k x加b，给定了两个点（一,一）和（二, 四），这两个点都在函数上，  一 一加一等于二'

In [24]:
import re
text = "!@#$%^&*()_-++[{]}\|;:''"",.<>?/！@#¥%……&*（）-——=+「【『「」』】、｜；：‘’“”，。《》？/]"
no_read = ''
for i in text:
    res = zh_normalizer.normalize(i)
    if not re.match(r'[\u4e00-\u9fff]', res):
        no_read += i
    else:
        print(res)

减
加
加
除以
减
等于
加
除以


In [25]:
no_read

"!@#$%^&*()_[{]}\\|;:'',.<>?！@#¥%……&*（）——「【『「」』】、｜；：‘’“”，。《》？]"

In [13]:
import re
from clients.utils import math_request

formular2text_url = 'http://123.57.25.89:8023/inference'

def remove_spaces_between_cn_en(text):
    punctuations = r'\.\,\?\:\!\，\。\；\：\、\！\？\"'
    text = re.sub(r'[ ]+', ' ', text)
    
    # 移除标点符号前后的空格
    text = re.sub(r'\s+([{}])'.format(re.escape(punctuations)), r'\1', text)
    #text = re.sub(r'([{}])\s+'.format(re.escape(punctuations)), r'\1', text)
    
    # 移除中文和英文字符之间的空格
    text = re.sub(r'([\u4e00-\u9fff]) (\w)', r'\1\2', text)
    text = re.sub(r'(\w) ([\u4e00-\u9fff])', r'\1\2', text)
    text = re.sub(r'[ ]?#9[ ]?', '#9', text)
    return text.strip(' ')

#fo = open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/qqx_latex_norm', 'w', encoding='utf8')

text_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/qqx_latex'
with open(text_file, 'r', encoding='utf8') as fin:
    for line in fin:
        try:
            text = line.strip()
            text = '$\Delta$'
            text = text.replace('\\\\','\\')
            detected_text = math_request(
                    text, formular2text_url, type='latex')
        
            detected_text = remove_spaces_between_cn_en(detected_text)
            #fo.write(f'{text}\t{detected_text}\n')
            print(detected_text)
            break
        except:
            print(text)
#fo.close()

Delta


In [23]:
chr( 22909)

'好'

In [30]:
import jieba
import jieba.posseg
import re

word = '一 行  白 鹭 上 青 天 '

word = re.sub(r' ', '', word)

word_list = jieba.posseg.cut(word)
res = []
for word, pos in word_list:
    res.append((word, pos))
print(res)

[('一行', 'm'), ('白鹭', 'nr'), ('上', 'f'), ('青天', 't')]


In [31]:
chr(19978)

'上'

In [25]:
from tal_frontend.frontend.g2p.bertg2pw.g2p_client import TAL_G2P_Triton
url = "112.126.23.219:80"
    
client = TAL_G2P_Triton(url)
sentencs = ["hello world"]
res, phonemes_id = client.infer(sentencs)
print(res)
print(phonemes_id)

/mnt/cfs/SPEECH/hupeng/tools/env/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


poly phonemes: ['HH EH0 L OW1']
hello
g2p bert res: [['hello world ', ['HH EH0 L OW1', 'W ER1 L D'], 'en']]
g2p res: ['HH EH0 L OW1', 'W ER1 L D']
HH EH0 L OW1
W ER1 L D


In [9]:
import re
rhys =  [0, 0, 0, 0, 0]

phonemes = ['R EH1 M B ER0', 'T UW1', 'B IY1', 'HH AE1 P IY0']

words = ['rember', 'to', 'be', 'happy', '!']

id2rhy = {1:'#0', 2:'#1', 3:'#2', 4:'#3'}
rhy_res = ['#0']
phonemes_res = []
for word, rhy in zip(words, rhys):
    if re.match(r'[\,\.\?\:\!，。；：、！？]', word):
        rhy_res.append('#3')
        tmp_rhy = ['#3']
        phonemes_res.append('sp3')
    elif rhy != 0:
        phoneme = phonemes.pop(0)
        phoneme = phoneme.split()
        tmp_rhy = ['0'] * (len(phoneme))
        tmp_rhy[-1] = id2rhy[rhy]
        rhy_res.extend(tmp_rhy)
        phonemes_res.extend(phoneme)
    else:
        if rhy_res[-1] == '#0':
            rhy_res[-1] = '#1'
        phoneme = phonemes.pop(0)
        phoneme = phoneme.split()
        tmp_rhy = ['0'] * (len(phoneme))
        tmp_rhy[-1] = '#1'
        rhy_res.extend(tmp_rhy)
        phonemes_res.extend(phoneme)
    print(word, ':', tmp_rhy)
rhy_res = rhy_res[1:]
print(rhy_res)
print(phonemes_res)

rember : ['0', '0', '0', '0', '#1']
to : ['0', '#1']
be : ['0', '#1']
happy : ['0', '0', '0', '#1']
! : ['#3']
['0', '0', '0', '0', '#1', '0', '#1', '0', '#1', '0', '0', '0', '#1', '#3']
['R', 'EH1', 'M', 'B', 'ER0', 'T', 'UW1', 'B', 'IY1', 'HH', 'AE1', 'P', 'IY0', 'sp3']


In [26]:
text = "你好"
match_en = re.match(r'^[a-zA-Z0-9]+', text)

if match_en:
    en_word = match_en.group(0)
    print(en_word)

In [10]:
a = ['#0', '0', '#1', '0', '#0', '0', '#1', '0', '#0', '0', '#1', '0', '#0', '0', '#0', '0', '#0', '0', '#1', '0', '#0', '0', '#1', '0', '0', '#1', '#3']
b = ['uo3', 'm', 'en5', 'x', 'u1', 'y', 'ao1', 'q', 'iou2', 'ch', 'u1', 'ch', 'ang2', 'f', 'ang1', 'x', 'ing2', 'd', 'e5', 'k', 'uan1', 'd', 'u4', 'EH1', 'K', 'S', 'sp3']
print(len(a), len(b))

27 27


In [22]:
from tal_frontend.frontend.ssml.xml2text import xml_reader, xml_reader_string
from tal_frontend.frontend.ssml.xml_processor import DomXml


text = '<speak voice_type="xiaosi" rate="1" pitch="1" volume="1" emotion="happy"><break time="3000ms"/><phoneme lang="en" ph="HH AH0 L OW1/W ER1 D">Hello word!</phoneme><break time="100ms"/><phoneme lang="cn" ph="yi4/huir2/jia1/ban1">一会儿加班。</phoneme><break time="1000ms"/>下面是latex格式的数学公式，<math interpret-as="latex">\\triangle OCM\cong \\triangle ABE</math><break time="1000ms"/>下面是asciimath格式的数学公式，<math interpret-as="asciimath">ax^2+bx+c=0</math>下面是mathml格式的数学公式，<math interpret-as="mathml"><mi>a</mi><mrow><msup><mrow><mi>x</mi></mrow><mrow><mn>2</mn></mrow></msup></mrow><mo>+</mo><mi>b</mi><mi>x</mi><mo>+</mo><mi>c</mi><mo>=</mo><mn>0</mn></math></speak>'

dom_processor = DomXml(text)

In [21]:
'好'.lower()

'好'

In [23]:
polyphonic_chars_path = "/mnt/cfs/NLP/hsy/audio/project/bert/g2pW/saved_models/step/v2/POLYPHONIC_CHARS_0713.txt"
polyphonic_chars = [line.split('\t') for line in open(polyphonic_chars_path).read().strip().split('\n')]

In [8]:
a = {'以':{'1':'1'}}
list(a)[0]

'以'

In [1]:
from tal_frontend.frontend.g2p_pp.g2p_pp_client import TAL_G2P_Triton

url = "123.56.235.205:80"
    
client = TAL_G2P_Triton(url)

sentences = ["一行白鹭上青天，hello world你好z世界"]

client.infer(sentences)

/mnt/cfs/SPEECH/hupeng/tools/env/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


IndexError: string index out of range

In [11]:
import re

text = 'They"re boos'

result = re.sub(r'([a-zA-Z]+)["]+([a-z]+)', r"\1'\2", text)

print(result)

They're boos


In [1]:
chr(13356)

'㐬'

In [2]:
ord('嗯')

21999

In [2]:
from tal_frontend.frontend.g2p_pp.g2p_pp_client import TAL_G2PPP_Triton

url = "123.56.235.205:80"
    
client = TAL_G2PPP_Triton(url)

/mnt/cfs/SPEECH/hupeng/tools/env/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


==>> self.bert_model_path: /mnt/cfs/NLP/hub_models/bert-base-multilingual-cased


In [23]:
# sentences = ['这道题要我们解决的问题是求一个等腰三角形的顶角的度']
sentences = ["回"]


rhy_res_l, phonemes_res_l = client.infer(sentences)
# print(rhy_res_l, phonemes_res_l)
print(phonemes_res_l)

[['h', 'ui2']]


In [14]:
import json

file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/10w_tts.json'
with open(file, 'r', encoding='utf8') as fin:
    fin = fin.read()
    fin = json.loads(fin)

In [24]:
texts = set()
for item in fin['RECORDS']:
    text = item['tts_output_text']
    if text not in texts:
        text = text.replace("\n", '')
        texts.add(text)

In [26]:
with open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/10w_tts_uniq.txt','w',encoding='utf8') as fo:
    count = 1
    for i in texts:
        fo.write(f'{str(count).zfill(5)}\t{i}\n')
        count += 1
fo.close()

In [1]:
import json
from tqdm import tqdm
from tal_frontend.tal_tts import Frontend
ft = Frontend()
ids, texts, nor_texts = [], [], []
fo = open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/10w_tts_norm.txt', 'w', encoding='utf8')

with open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/10w_tts_uniq.txt','r',encoding='utf8') as fin:
    lines = fin.readlines()
    for i in tqdm(range(len(lines))):
        line = lines[i]
        try:
            key, text = line.strip().split('\t', maxsplit=1)
            nor_text = ft.process_text_tn(text)
            nor_text = ' '.join([i[0] for i in nor_text])
            fo.write(f'{key}\t{text}\t{nor_text}\n')
        except:
            pass
fo.close()

100%|██████████| 34593/34593 [19:06<00:00, 30.17it/s] 


In [4]:
import torchaudio
'''
e9326e4bb84a622e_1	/mnt/cfs/SPEECH/data/tts/Audio_Book/part_00/seperate_data/data/book_0000/e9326e4bb84a622e.mp3	3.4375	15.46875	12.03125
'''
file = r'/mnt/cfs/SPEECH/data/tts/Audio_Book/part_00/seperate_data/data/book_0000/e9326e4bb84a622e.mp3'
fs = 441000
st, et = 3.4375, 15.46875

audio = torchaudio.load(file, frame_offset=st*fs, num_frames=et*fs, normalize=True)

ModuleNotFoundError: No module named 'torchaudio'

In [1]:
from tal_frontend.frontend.g2p_pp.g2p_pp_client import TAL_G2PPP_Triton

url = "123.56.235.205:80"
    
client = TAL_G2PPP_Triton(url)

# sentences = ['这道题要我们解决的问题是求一个等腰三角形的顶角的度']
sentences = [
"i like eat banana, how about you?"]


rhy_res_l, phonemes_res_l = client.infer(sentences)
print(rhy_res_l, phonemes_res_l)


TypeError: TAL_G2PPP_Triton.__init__() missing 8 required positional arguments: 'model_name', 'tokenizer', 'labels', 'char2phonemes', 'en_monophone', 'tonesandhi', 'zh_front', and 'en_frontend'

In [4]:
from tal_frontend.frontend.g2p.zh_frontend import Frontend as zhFrontend
import re

zh_front = zhFrontend()
words = ['刀刀见血十']

for w in words:
    ph = zh_front.get_phonemes(w)
    print(f'{w} {ph}')
xue = re.findall(r'x ve4|x ie3', ph)
print(xue)

刀刀见血十 d ao1 d ao1 j ian4 x ie3 sh iii2
['x ie3']


In [15]:
xue.pop(0)

'x ve4'

In [14]:
xue

['x ve4', 'x ve4', 'x ie3']

In [1]:
from tal_frontend.frontend.g2p.en_frontend import English
en_frontend = English()

In [2]:
word = 'hello'
en_frontend.phoneticize(word)

'HH AH0 L OW1'

In [20]:
import re

text = '（2,-4），'

clean_text = re.sub(r'([\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\]\\\^\_\`\{\|\}\~～！，。？《》/：；”“’‘【】「」！¥……（）——]{2})[\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/:;<=>\?\@\[\]\\\^_`{|}~～！，。？《》/：；”“’‘【】「」！¥……（）——]+', r'\1', text)

# clean_text = re.sub(r'([\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\]\\\^\_\`\{\|\}\~ ～！，。？《》/：；”“’‘【】「」！¥……（）——]){2,}', r'\1', text)

print(clean_text)

（2,-4），


In [1]:
import re
fo = open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/3kw_query_diff.csv', 'w', encoding='utf8')
with open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/3kw_query.csv', 'r', encoding='utf8') as fin:
    for line in fin:
        try:
            utt, ori, tal, zijie = line.strip().split('\t')
            clean_zijie = zijie.lower()
            if clean_zijie == '#n/a':
                continue
            clean_tal = tal.lower()
            clean_zijie = re.findall(r'[a-z\u4e00-\u9fff]', clean_zijie)
            clean_tal = re.findall(r'[a-z\u4e00-\u9fff]', clean_tal)
            clean_tal = ''.join(clean_tal)
            clean_zijie = ''.join(clean_zijie)
            if '湿度' in clean_tal:
                clean_tal = re.sub('百分之', '', clean_tal)
            if clean_zijie != clean_tal and clean_zijie and clean_tal:
                
                tal = re.sub(r'/', '', tal)
                zijie = re.sub(r'#\d', '', zijie)
                fo.write(f'{utt}\t{ori}\t{tal}\t{zijie}\n')
        except:
            pass
fo.close()

In [9]:
from tal_frontend.frontend.normalizer.cn.cn_normalizer import cn_Normalizer as cn_Normalizer
zh_normalizer = cn_Normalizer()
text = '舞剑歌 ，唐， 李靖 ，陟崇冈兮望四围，□□闪□兮断虹飞，嗟嗟三军唱凯归。'
nor_text = zh_normalizer.normalize(text)
print(nor_text)

舞剑歌 ，唐， 李靖 ，陟崇冈兮望四围，□□闪□兮断虹飞，嗟嗟三军唱凯归。


In [8]:
ord('忄')

24516

In [ ]:
import json
import re
from tal_frontend.frontend.g2p.zh_frontend import Frontend as zhFrontend
from tal_frontend.frontend.g2p.bertg2pw.pinyin_dict import cn_monophone
from tal_frontend.frontend.g2p.bertg2pw.tal_dict import tal_cn_dict

def phoneme_convertor(phoneme):
    if phoneme.isupper():  
        return phoneme
    else:
        if phoneme == 'xer':
            return 'er2'
        else:
            tone = phoneme[-1]
            phoneme = phoneme[:-1]
            phoneme = tal_cn_dict[phoneme] + tone
            return phoneme

zh_front = zhFrontend()


for word, ph in cn_monophone.items():
    try:
        ph = phoneme_convertor(ph)
        ph = re.sub(r'\s', '', ph)
    except:
        pass
    g2p = zh_front.get_phonemes(chr(word))
    g2p = re.sub(r'\s', '', g2p)
    if g2p != ph:
        print(chr(word), g2p, ph)


In [1]:
import json
import re
from collections import defaultdict

file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/10w_tts.json'
with open(file, 'r') as fin:
    fin = fin.read()
    fin = json.loads(fin)

used_id = set()
for item in fin['RECORDS']:
    query_trace_id = item['query_trace_id']
    used_id.add(query_trace_id)
file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/query_data_3kw.json'

with open(file, 'r') as fin:
    fin = fin.read()
    fin = json.loads(fin)

In [2]:
res = set()
for item in fin['RECORDS']:
    text = item['tts_output_text']
    if text not in res:
        res.add(text)
print(len(res))

937419


In [135]:
text_dict = defaultdict(list)
for item in fin['RECORDS']:
    text = item['tts_output_text']
    query_trace_id = item['query_trace_id']
    if query_trace_id in used_id:
        continue
    if not text:
        continue
    text = re.sub(r'.*<.*>.*|https.*|\n', '', text)
    clean_text = re.sub(r'.*</.*>.*|https.*|[\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/:;<=>\?\@\[\]\\\^_`{|}~\s！，。？《》/：；”“’‘【】「」！¥……（）——\da-zA-Z]', '', text)
    clean_text = re.sub(r'(.*)(已为您找到|为你找到|意思是|找到多个|这样写|找到以下)(.*)',r'\2',clean_text)
    if not clean_text:
        continue
    if text_dict.get(clean_text):
        text_dict[clean_text][1] += 1
    else:
        text_dict[clean_text] = [text, 1, query_trace_id]
sorted_data = sorted(text_dict.items(), key=lambda item: item[1][1], reverse=True)

num1, num2 = 0, 0
fo1 = open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/uniq_3kw_data.list','w',encoding='utf8')
fo2 = open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/test_data/text/uniq_3kw_1_data.list','w',encoding='utf8')

for data in sorted_data:
    clean_text = data[0]
    ori_text = data[1][0]
    count = data[1][1]
    query_trace_id = data[1][2]
    if count > 1:
        num1 += 1
        fo1.write(f'{query_trace_id}\t{clean_text}\t{ori_text}\t{count}\n')
    elif count == 1:
        num2 +=1 
        fo2.write(f'{query_trace_id}\t{clean_text}\t{ori_text}\t{count}\n')
            
fo1.close()
fo2.close()
print(len(sorted_data))
print(num1)
print(num1 / 1.5 / 3600 / 24)
print(num2)

176289
62020
0.47854938271604935
114269


In [134]:
text = '的意思是：决定；决心；决议；结论；裁决；判定.'
text = re.sub(r'.*</.*>.*|https.*|\n', '', text)
print(text)
clean_text = re.sub(r'.*</.*>.*|https.*|[\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/:;<=>\?\@\[\]\\\^_`{|}~\s！，。？《》/：；”“’‘【】「」！¥……（）——\da-zA-Z]', '', text)

clean_text = re.sub(r'(.*)(已为您找到|为你找到|意思是|找到多个|这样写|找到以下)(.*)',r'\2',clean_text)

print(clean_text)

的意思是：决定；决心；决议；结论；裁决；判定.
意思是


In [9]:
import re
from tal_frontend.utils.special_words import special_words
text = '竖心旁和心理活动或心情有关。【忄】作为部首叫：竖心旁【忄】读音： xīn ，意思是：同“心”，用作偏旁。俗称“竖心旁”，简称“竖心”。【忄】的笔顺：点、点、竖含有【忄】的字：怡、惨、憾、忧、悦'
for key, value in special_words.items():
    text = re.sub(key, value, text)
print(text)

word = 'NWC'
if re.match(r'[a-zA-Z]', word):
    print(word)

竖心旁和心理活动或心情有关。【忄】作为部首叫：竖心旁【忄】读音： 西因心 ，意思是：同“心”，用作偏旁。俗称“竖心旁”，简称“竖心”。【忄】的笔顺：点、点、竖含有【忄】的字：怡、惨、憾、忧、悦
NWC


In [13]:
from tal_frontend.frontend.g2p.en_frontend import English

en = English()
en.phoneticize('igl')

'IH1 G AH0 L'

In [26]:
import re
nor_text = '全文是这样的：,```你好```这首诗'
nor_text = re.sub(r'([：,`])[,`]+', r'\1 ', nor_text)
print(nor_text)

word = "you're"
if re.match(r'[a-zA-Z]', word):
    print(1)
    
text = "you----re"
text = re.sub(r'([a-zA-Z]+)[\'\-]+([a-zA-Z]+)',r'\1\2',text)
print(text)

text = "girl.mr.Dr"
sentence = re.sub(r'(mr|Mr|Ms|ms|dr|Dr)\.', r'\1', text)
print(sentence)

全文是这样的： 你好` 这首诗
1
youre
girl.mrDr


In [40]:
import jieba

text = "you're beatiful, a good-looking girl"

jieba.lcut(text)

['you',
 "'",
 're',
 ' ',
 'beatiful',
 ',',
 ' ',
 'a',
 ' ',
 'good',
 '-',
 'looking',
 ' ',
 'girl']

In [37]:
len("you're beatiful, a good-looking girl")

36

In [39]:
a =  ['youre', ' ', 'beatiful', ',', 'a', ' ', 'good', ' ', 'looking', ' ', 'girl']
for i in a:
    if re.search(r'[a-z]+', i):
        print(i)

youre
beatiful
a
good
looking
girl


In [2]:
from tal_frontend.frontend.ssml.xml2text import xml_reader, xml_reader_string

ssml = '<speak voice_type=\"\" rate=\"1.1\" pitch=\"1.0\" volume=\"90\" emotion=\"cheerful\">整顿衣<phoneme lang=\"cn\" ph=\"shang5\">裳</phoneme>起敛容。</speak>'

contents, speak_info = xml_reader_string(ssml)


In [3]:
print(contents)

[['text', '', '整顿衣'], ['pinyin', 'sh ang5/', '裳'], ['text', '', '起敛容。']]


In [9]:
import redis
import json


# prod
redis_client = redis.Redis(host="r-2ze7sx5pe00o4w0mer.redis.rds.aliyuncs.com",
                           port=6379,
                           password="DySfiNWD9Ilwa91_",
                           db=8)

# ['整 顿 衣 裳 起 敛 容 ', ['zh eng3', 'd uen4', 'i1', 'sh ang5', 'q i3', 'l ian3', 'r ong2'], 'cn']

# write_key = "整顿衣裳起敛容"
# write_value = json.dumps(["zheng3 duen4 i1 shang5 qi3 lian3 rong2","整顿衣裳起敛容#4"],ensure_ascii=False)
# redis_client.set(write_key,write_value)

# res = redis_client.get(write_key)
# print(json.loads(res))

In [6]:
from tal_frontend.frontend.g2p.tone_sandhi import ToneSandhi

ts = ToneSandhi()

text = '北 京 有 很 多 有 趣 的 地 方 哦 '

phs = ['b ei3', 'j ing1', 'sp3', 'iou3', 'h en3', 'd uo1', 'iou3', 'q v4', 'd e5', 'd i4', 'f ang1', 'o5']

res = ts.modified_tone(text, phs)
print(res)

text = '北 京 有 很 多 有 趣 的 地 方 哦 '

phs = ['b ei3', 'j ing1',  'iou3', 'h en3', 'd uo1', 'iou3', 'q v4', 'd e5', 'd i4', 'f ang1', 'o5']

res = ts.modified_tone(text, phs)
print(res)

['b ei3', 'j ing1', ' sp3', ' iou2', 'h en3', 'd uo1', ' iou3', 'q v5', 'd e5', 'd i5', 'f ang5']
['b ei3', 'j ing1', ' iou3', 'h en3', 'd uo1', ' iou3', 'q v4', 'd e5', 'd i4', 'f ang5', ' o5']


In [43]:
elements = {
    "H": {"中文名": "氢", "拼音": "qīng"},
    "He": {"中文名": "氦", "拼音": "hài"},
    "Li": {"中文名": "锂", "拼音": "lǐ"},
    "Be": {"中文名": "铍", "拼音": "pí"},
    "B": {"中文名": "硼", "拼音": "péng"},
    "C": {"中文名": "碳", "拼音": "tàn"},
    "N": {"中文名": "氮", "拼音": "dàn"},
    "O": {"中文名": "氧", "拼音": "yǎng"},
    "F": {"中文名": "氟", "拼音": "fú"},
    "Ne": {"中文名": "氖", "拼音": "nǎi"},
    "Na": {"中文名": "钠", "拼音": "nà"},
    "Mg": {"中文名": "镁", "拼音": "měi"},
    "Al": {"中文名": "铝", "拼音": "lǚ"},
    "Si": {"中文名": "硅", "拼音": "guī"},
    "P": {"中文名": "磷", "拼音": "lín"},
    "S": {"中文名": "硫", "拼音": "liú"},
    "Cl": {"中文名": "氯", "拼音": "lǜ"},
    "Ar": {"中文名": "氩", "拼音": "yà"},
    "K": {"中文名": "钾", "拼音": "jiǎ"},
    "Ca": {"中文名": "钙", "拼音": "gài"},
    "Sc": {"中文名": "钪", "拼音": "kàng"},
    "Ti": {"中文名": "钛", "拼音": "tài"},
    "V": {"中文名": "钒", "拼音": "fán"},
    "Cr": {"中文名": "铬", "拼音": "gè"},
    "Mn": {"中文名": "锰", "拼音": "měng"},
    "Fe": {"中文名": "铁", "拼音": "tiě"},
    "Co": {"中文名": "钴", "拼音": "gǔ"},
    "Ni": {"中文名": "镍", "拼音": "niè"},
    "Cu": {"中文名": "铜", "拼音": "tóng"},
    "Zn": {"中文名": "锌", "拼音": "xīn"},
    "Ga": {"中文名": "镓", "拼音": "jiā"},
    "Ge": {"中文名": "锗", "拼音": "zhě"},
    "As": {"中文名": "砷", "拼音": "shēn"},
    "Se": {"中文名": "硒", "拼音": "xī"},
    "Br": {"中文名": "溴", "拼音": "xiù"},
    "Kr": {"中文名": "氪", "拼音": "kè"},
    "Rb": {"中文名": "铷", "拼音": "rú"},
    "Sr": {"中文名": "锶", "拼音": "sī"},
    "Y": {"中文名": "钇", "拼音": "yǐ"},
    "Zr": {"中文名": "锆", "拼音": "gào"},
    "Nb": {"中文名": "铌", "拼音": "ní"},
    "Mo": {"中文名": "钼", "拼音": "mù"},
    "Tc": {"中文名": "锝", "拼音": "dé"},
    "Ru": {"中文名": "钌", "拼音": "liǎo"},
    "Rh": {"中文名": "铑", "拼音": "lǎo"},
    "Pd": {"中文名": "钯", "拼音": "bǎ"},
    "Ag": {"中文名": "银", "拼音": "yín"},
    "Cd": {"中文名": "镉", "拼音": "gé"},
    "In": {"中文名": "铟", "拼音": "yīn"},
    "Sn": {"中文名": "锡", "拼音": "xī"},
    "Sb": {"中文名": "锑", "拼音": "tī"},
    "Te": {"中文名": "碲", "拼音": "dì"},
    "I": {"中文名": "碘", "拼音": "diǎn"},
    "Xe": {"中文名": "氙", "拼音": "xiān"},
    "Cs": {"中文名": "铯", "拼音": "sè"},
    "Ba": {"中文名": "钡", "拼音": "bèi"},
    "La": {"中文名": "镧", "拼音": "lán"},
    "Ce": {"中文名": "铈", "拼音": "shì"},
    "Pr": {"中文名": "镨", "拼音": "pǔ"},
    "Nd": {"中文名": "钕", "拼音": "nǚ"},
    "Pm": {"中文名": "钷", "拼音": "pǒ"},
    "Sm": {"中文名": "钐", "拼音": "shān"},
    "Eu": {"中文名": "铕", "拼音": "yǒu"},
    "Gd": {"中文名": "钆", "拼音": "gá"},
    "Tb": {"中文名": "铽", "拼音": "tè"},
    "Dy": {"中文名": "镝", "拼音": "dī"},
    "Ho": {"中文名": "钬", "拼音": "huǒ"},
    "Er": {"中文名": "铒", "拼音": "ěr"},
    "Tm": {"中文名": "铥", "拼音": "diū"},
    "Yb": {"中文名": "镱", "拼音": "yì"},
    "Lu": {"中文名": "镥", "拼音": "lǔ"},
    "Hf": {"中文名": "铪", "拼音": "hā"},
    "Ta": {"中文名": "钽", "拼音": "tǎn"},
    "W": {"中文名": "钨", "拼音": "wū"},
    "Re": {"中文名": "铼", "拼音": "lái"},
    "Os": {"中文名": "锇", "拼音": "é"},
    "Ir": {"中文名": "铱", "拼音": "yī"},
    "Pt": {"中文名": "铂", "拼音": "bó"},
    "Au": {"中文名": "金", "拼音": "jīn"},
    "Hg": {"中文名": "汞", "拼音": "gǒng"},
    "Tl": {"中文名": "铊", "拼音": "tā"},
    "Pb": {"中文名": "铅", "拼音": "qiān"},
    "Bi": {"中文名": "铋", "拼音": "bì"},
    "Po": {"中文名": "钋", "拼音": "pō"},
    "At": {"中文名": "砹", "拼音": "ài"},
    "Rn": {"中文名": "氡", "拼音": "dōng"},
    "Fr": {"中文名": "钫", "拼音": "fāng"},
    "Ra": {"中文名": "镭", "拼音": "léi"},
    "Ac": {"中文名": "锕", "拼音": "ā"},
    "Th": {"中文名": "钍", "拼音": "tǔ"},
    "Pa": {"中文名": "镤", "拼音": "pú"},
    "U": {"中文名": "铀", "拼音": "yóu"},
    "Np": {"中文名": "镎", "拼音": "ná"},
    "Pu": {"中文名": "钚", "拼音": "bù"},
    "Am": {"中文名": "镅", "拼音": "méi"},
    "Cm": {"中文名": "锔", "拼音": "jú"},
    "Bk": {"中文名": "锫", "拼音": "péi"},
    "Cf": {"中文名": "锎", "拼音": "kāi"},
    "Es": {"中文名": "锿", "拼音": "āi"},
    "Fm": {"中文名": "镄", "拼音": "fèi"},
    "Md": {"中文名": "钔", "拼音": "mén"},
    "No": {"中文名": "锘", "拼音": "nuò"},
    "Lr": {"中文名": "铹", "拼音": "láo"},
    "Mc": {"中文名": "镆", "拼音": "mò"},
}

for key, value in elements.items():
    name = value['中文名']
    a = f'{name}-\t{name}'
    print(a)

氢-	氢
氦-	氦
锂-	锂
铍-	铍
硼-	硼
碳-	碳
氮-	氮
氧-	氧
氟-	氟
氖-	氖
钠-	钠
镁-	镁
铝-	铝
硅-	硅
磷-	磷
硫-	硫
氯-	氯
氩-	氩
钾-	钾
钙-	钙
钪-	钪
钛-	钛
钒-	钒
铬-	铬
锰-	锰
铁-	铁
钴-	钴
镍-	镍
铜-	铜
锌-	锌
镓-	镓
锗-	锗
砷-	砷
硒-	硒
溴-	溴
氪-	氪
铷-	铷
锶-	锶
钇-	钇
锆-	锆
铌-	铌
钼-	钼
锝-	锝
钌-	钌
铑-	铑
钯-	钯
银-	银
镉-	镉
铟-	铟
锡-	锡
锑-	锑
碲-	碲
碘-	碘
氙-	氙
铯-	铯
钡-	钡
镧-	镧
铈-	铈
镨-	镨
钕-	钕
钷-	钷
钐-	钐
铕-	铕
钆-	钆
铽-	铽
镝-	镝
钬-	钬
铒-	铒
铥-	铥
镱-	镱
镥-	镥
铪-	铪
钽-	钽
钨-	钨
铼-	铼
锇-	锇
铱-	铱
铂-	铂
金-	金
汞-	汞
铊-	铊
铅-	铅
铋-	铋
钋-	钋
砹-	砹
氡-	氡
钫-	钫
镭-	镭
锕-	锕
钍-	钍
镤-	镤
铀-	铀
镎-	镎
钚-	钚
镅-	镅
锔-	锔
锫-	锫
锎-	锎
锿-	锿
镄-	镄
钔-	钔
锘-	锘
铹-	铹
镆-	镆
